<h3>things to import</h3>

In [1]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
from copy import copy

<h3>catalogue file for all SPI sources</h3>

In [2]:
allcat = fits.open('spi_cat_all_source_spimodfit.fits.gz')

<h3>extensions</h3>

In [3]:
allcat.info()

Filename: spi_cat_all_source_spimodfit.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   ()      
  1  SPI.-SRCL-CAT    1 BinTableHDU    112   256R x 36C   [16A, D, 19A, I, E, E, E, B, B, B, B, 32A, I, 8E, 128A, I, 32E, 128A, I, 8E, 24A, E, E, E, E, E, E, E, E, 256E, 256E, 256E, 256E, I, I, 19A]   


<h3>columns in extension 1</h3>

In [4]:
allcat[1].columns

ColDefs(
    name = 'BIRD_SOURCE_ID'; format = '16A'
    name = 'DAY_ID'; format = 'D'; unit = 'd'
    name = 'NAME'; format = '19A'
    name = 'CLASS'; format = 'I'; bscale = 1; bzero = 32768
    name = 'RA_OBJ'; format = 'E'; unit = 'deg'
    name = 'DEC_OBJ'; format = 'E'; unit = 'deg'
    name = 'ERR_RAD'; format = 'E'; unit = 'deg'
    name = 'ISGRI_FLAG'; format = 'B'
    name = 'JEMX_FLAG'; format = 'B'
    name = 'SPI_FLAG'; format = 'B'
    name = 'PICSIT_FLAG'; format = 'B'
    name = 'SPA_MODL'; format = '32A'
    name = 'SPA_NPAR'; format = 'I'; bscale = 1; bzero = 32768
    name = 'SPA_PARS'; format = '8E'
    name = 'SPE_MODL'; format = '128A'
    name = 'SPE_NPAR'; format = 'I'; bscale = 1; bzero = 32768
    name = 'SPE_PARS'; format = '32E'
    name = 'VAR_MODL'; format = '128A'
    name = 'VAR_NPAR'; format = 'I'; bscale = 1; bzero = 32768
    name = 'VAR_PARS'; format = '8E'
    name = 'COMMENTS'; format = '24A'
    name = 'SPI_FLUX_1'; format = 'E'; unit = 'ph/cm2/s'

<h3>to make a new entry, these columns are required for your new sources</h3>

<h3>Suppose your new source is at the position $(l,b) = (180.00,-0.00)$ and has the name Galactic Anti-Center. Spimodfit requires the source to be in RA and DEC and not in Galactic longitude and latitude. Therefore, we convert the coordinates to RA and DEC:</h3>

In [19]:
gl = 180.00
gb = -0.00

In [20]:
coords = SkyCoord(l=gl,b=gb,unit='deg',frame='galactic')

In [21]:
ra,dec = coords.icrs.ra.value,coords.icrs.dec.value

In [22]:
ra,dec

(86.40498828654475, 28.93617776179148)

<h3>the name should be meaningful:</h3>

In [23]:
name = 'Gal_AntiCenter'

<h3>to append a row in the table, we need to know how many are in there right now:</h3>

In [24]:
n_src = len(allcat[1].data)
n_src

256

In [25]:
new_cat = fits.BinTableHDU.from_columns(allcat[1].columns, nrows=n_src+1)

<h3>now we can the same entries, except that the last one is empty</h3>

In [26]:
new_cat.data['NAME'][-1]

''

<h3> to simplyfy the procedure, we copy the entries from the second-to-last to the last:</h3>

In [27]:
new_cat.data[-1] = copy(new_cat.data[-2])

In [28]:
print(new_cat.data['NAME'][-1])
print(new_cat.data['SOURCE_ID'][-1])
print(new_cat.data['RA_OBJ'][-1])
print(new_cat.data['DEC_OBJ'][-1])

V2400 Oph
V2400 Oph
258.174
-24.268105


<h3>we need to change the NAME, RA_OBJ, DEC_OBJ, and SOURCE_ID column for our new source:</h3>

In [29]:
new_cat.data['NAME'][-1] = name
new_cat.data['SOURCE_ID'][-1] = name
new_cat.data['RA_OBJ'][-1] = ra
new_cat.data['DEC_OBJ'][-1] = dec

In [30]:
print(new_cat.data['NAME'][-1])
print(new_cat.data['SOURCE_ID'][-1])
print(new_cat.data['RA_OBJ'][-1])
print(new_cat.data['DEC_OBJ'][-1])

Gal_AntiCenter
Gal_AntiCenter
86.40499
28.936178


<h3>Finally, we need to create a new fits file and save it:</h3>

In [31]:
new_cat.name = 'SPI.-SRCL-CAT'

In [32]:
new_cat.writeto('spi_cat_all_source_GalAntiCenter_spimodfit.fits',overwrite=True)